In [1]:
%load_ext aiida
%aiida

Loaded AiiDA DB environment - profile name: c2sm.

In [2]:
import io
import pendulum
import pathlib
import f90nml
import tempfile
from aiida import orm, engine
from aiida_c2sm import spice

In [3]:
def get_ecraddir() -> orm.RemoteData:
    try:
        params = orm.load_node(label="spice-ecraddir")
        return params
    except common.NotExistent:
        return orm.RemoteData(
            label="spice-ecraddir",
            computer=orm.load_computer("Daint"),
            remote_path="/scratch/snx3000/mjaehn/sandbox_workflow/spice/icon-nwp-gpu/externals/ecrad/data",
        ).store()

In [4]:
builder = spice.gcm2icon.workflow.Gcm2Icon.get_builder()

In [5]:
src_dir = pathlib.Path("/scratch/snx3000/mjaehn/sandbox_workflow/spice/src")

builder.boundary_data = spice.data.get_inidata()
builder.ecraddir = get_ecraddir()
builder.gcm_data = spice.data.get_gcm_data()
builder.ini_basedir = spice.data.get_inibasedir()
builder.inidata = spice.data.get_inidata()

builder.computer_options = {
    "account": "csstaff",
    "queue_name": "normal",
    "custom_scheduler_commands": "#SBATCH -C gpu",
    "max_memory_kb": int(64e6),
    "resources": {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 1
    },
}

builder.prep.code = orm.load_code("spice-prep-installed")
builder.prep.computer_options = {
    "max_wallclock_seconds": 1800,
}

builder.conv.code = orm.load_code("spice-conv-installed")
builder.conv.computer_options = {
    "max_wallclock_seconds": 1800,
}

builder.icon.code = orm.load_code("spice-icon")
builder.icon.computer_options = {
    "max_wallclock_seconds": 3600,
    "resources": {
        "num_machines": 10,
        "num_mpiprocs_per_machine": 1,
        "num_cores_per_mpiproc": 2,
    },
    "withmpi": True,
    "mpirun_extra_params": [
        "--ntasks-per-node",
        "1",
        "--hint=nomultithread",
        "--cpus-per-task",
        "1",
    ],
}

builder.parameters = orm.JsonableData(
    spice.params.SpiceParams(
        start_date=pendulum.datetime(year=1979, month=1, day=1, tz="utc"),
        stop_date=pendulum.datetime(year=1979, month=3, day=1, tz="utc"),
        utils_bindir=src_dir / "utils" / "bin",
        cfu_bindir=src_dir / "cfu" / "bin",
        lam_grid_relpath="europe044/europe044_DOM01.nc",
        parent_grid_relpath="europe044/europe044_DOM01.parent.nc",
        extpar_relpath="europe044/external_parameter_icon_europe044_DOM01_tiles.nc",
        ghg_file_relpath="greenhouse_gases/bc_greenhouse_rcp45_1765-2500.nc"
    )
)

In [6]:
result = engine.run(builder)

08/22/2023 03:08:12 PM <38583> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [3634|Gcm2Icon|check_inputs]: Checking inputs.
08/22/2023 03:08:12 PM <38583> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [3634|Gcm2Icon|init_iterations]: Initializing iteration variables.
08/22/2023 03:08:12 PM <38583> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [3634|Gcm2Icon|prep]: Starting Preparation run for date 1979-01-01 00:00:00.
08/22/2023 03:21:02 PM <38583> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [3634|Gcm2Icon|conv]: Starting Conv2Icon run for date 1979-01-01 00:00:00.
08/22/2023 03:26:04 PM <38583> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [3634|Gcm2Icon|wait_for_previous_icon]: Making the next step wait for the previous Icon run.
08/22/2023 03:26:04 PM <38583> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [3634|Gcm2Icon|icon]: Starting Icon run for date 1979

In [7]:
result

{'remote_folder': <RemoteData: uuid: f74af39c-93c9-4604-9b1b-1f70903bda37 (pk: 3691)>,
 'retrieved': <FolderData: uuid: 5d1dbc7a-5f1b-42af-8131-b69d8d04b77f (pk: 3692)>}

In [8]:
result["remote_folder"].creator.pk

3690